# DOPP Ex 2 - Question 28 - Food in the EU

## 1. Intro

In [ ]:
Add our names, student numbers etc

Maybe do some intro to the topic?

introduce datasets, challenges finding the sets, then explain we had to change some datasets after the first discussion do to issues

## 2. Preprocessing datasets

### 2.1 Loading and cleaning import/export dataset

The Import/Export data is available for use in separate files, each file contains the data for EU conuntries for a separate year. Therefore the first step is to merge these files into a dataset. Since they all follow the same format, this is not a complicated task.

In [41]:
import pandas as pd

#Load all files into a separate df
df_trade = pd.read_csv('./raw/Exports and Imports/FAOSTAT_data_en_1-26-2025.csv')

In order to answer our defined questions, the best format for the data would be a df where each country has a row for each year and each food category is in a separate column. This way we can make more meaningful data exploration and the data would be in a format that is more suitable to analyze the changes within countries over the years and also between countries over the years. Another interesting topic could be differences based on geography. For example to compare countries with access and without access to see. Since this study is only focusing on food, we can drop the Domain column as it holds no new information to us. 

In [49]:
import pandas as pd

# Create the pivot table with multi-index columns so that each item gets a separate column with four values
df_pivot_trade = pd.pivot_table(
    df_trade,
    values='Value',
    index=['Year', 'Area'],
    columns=['Item', 'Element'],
    aggfunc='first',
    fill_value=0
)

# If you don't need the multi-index, jsut uncomment this and run the cell again
# Reset index to make Year and Area regular columns
# df_pivot_trade = df_pivot_trade.reset_index()

df_pivot_trade.to_csv('./data/processed_trade_data.csv', index=False)
df_pivot_trade

Item                                                    Alcoholic Beverages  \
Element                                                     Export quantity   
Year Area                                                                     
2000 Albania                                                          26.00   
     Austria                                                       90143.94   
     Belarus                                                       57176.81   
     Belgium                                                      525388.75   
     Bosnia and Herzegovina                                          806.00   
...                                                                     ...   
2023 Spain                                                       2781001.59   
     Sweden                                                       267489.67   
     Switzerland                                                    5991.15   
     Ukraine                                                      115735.45   
     United Kingdom of Great Britain and Northern Ir...          1531012.85   

Item                                                                  \
Element                                                 Export value   
Year Area                                                              
2000 Albania                                                   35.00   
     Austria                                                77874.50   
     Belarus                                                29353.16   
     Belgium                                               598481.10   
     Bosnia and Herzegovina                                   662.00   
...                                                              ...   
2023 Spain                                                4724103.85   
     Sweden                                                699991.35   
     Switzerland                                           126902.83   
     Ukraine                                               103607.60   
     United Kingdom of Great Britain and Northern Ir...   9758392.16   

Item                                                                     \
Element                                                 Import quantity   
Year Area                                                                 
2000 Albania                                                   33354.00   
     Austria                                                  128485.54   
     Belarus                                                   29365.93   
     Belgium                                                  516822.85   
     Bosnia and Herzegovina                                    44800.00   
...                                                                 ...   
2023 Spain                                                    877694.66   
     Sweden                                                   391660.29   
     Switzerland                                              294902.31   
     Ukraine                                                  232922.00   
     United Kingdom of Great Britain and Northern Ir...      2572362.02   

Item                                                                  \
Element                                                 Import value   
Year Area                                                              
2000 Albania                                                12533.00   
     Austria                                               207982.98   
     Belarus                                                41766.63   
     Belgium                                              1001198.63   
     Bosnia and Herzegovina                                 20000.00   
...                                                              ...   
2023 Spain                                                1999216.03   
     Sweden                                               1270817.05   
     Switzerland                         

In [51]:
# Reset index to make Year and Area regular columns
df_pivot_trade_reset = df_pivot_trade.reset_index()

# Get unique countries
unique_countries = df_pivot_trade_reset['Area'].unique()
unique_countries

array(['Albania', 'Austria', 'Belarus', 'Belgium',
       'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Czechia',
       'Denmark', 'Estonia', 'Faroe Islands', 'Finland', 'France',
       'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy',
       'Latvia', 'Lithuania', 'Luxembourg', 'Malta',
       'Netherlands (Kingdom of the)', 'North Macedonia', 'Norway',
       'Poland', 'Portugal', 'Republic of Moldova', 'Romania',
       'Russian Federation', 'Serbia and Montenegro', 'Slovakia',
       'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Ukraine',
       'United Kingdom of Great Britain and Northern Ireland',
       'Montenegro', 'Serbia'], dtype=object)

### 2.2 Loading and cleaning life expectancy dataset

In [76]:
#Load all files into a separate df
df_life = pd.read_csv('./raw/Life Expectancy/demo_mlexp.csv', encoding='cp1252', sep=';', na_values=[''])
df_life.head()

,TIME,2000,Unnamed: 2,2001,Unnamed: 4,2002,Unnamed: 6,2003,Unnamed: 8,2004,...,2019,Unnamed: 40,2020,Unnamed: 42,2021,Unnamed: 44,2022,Unnamed: 46,2023,Unnamed: 48
0,GEO (Labels),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Belgium,77.9,NaN,78.1,NaN,78.2,NaN,78.3,NaN,79.0,...,82.1,NaN,80.8,NaN,81.9,NaN,81.8,NaN,82.5,ip
2,Bulgaria,71.6,NaN,71.9,NaN,72.1,NaN,72.3,NaN,72.5,...,75.1,NaN,73.3,NaN,71.2,NaN,74.2,b,75.8,ip
3,Czechia,75.1,NaN,75.3,NaN,75.4,NaN,75.3,NaN,75.9,...,79.3,NaN,78.3,NaN,77.2,b,79.0,NaN,80.0,ip
4,Denmark,76.9,NaN,77.0,NaN,77.1,NaN,77.4,NaN,77.8,...,81.5,NaN,81.6,NaN,81.5,NaN,81.3,NaN,81.9,ip


Now let's clean up the data. As a first step elt's remove the unnecessary empty columns and any countries that are not in Europe.

In [77]:
# Drop unnamed columns and the first row (GEO Labels)
df_life = df_life.loc[:, ~df_life.columns.str.contains('Unnamed')]
df_life = df_life.iloc[1:]  # Remove first row

# Filter for specified countries 
countries = ['Albania', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 
           'Bulgaria', 'Croatia', 'Czechia', 'Denmark', 'Estonia', 'Faroe Islands',
           'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland',
           'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 
           'Netherlands (Kingdom of the)', 'North Macedonia', 'Norway', 'Poland',
           'Portugal', 'Republic of Moldova', 'Romania', 'Russian Federation',
           'Serbia and Montenegro', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
           'Switzerland', 'Ukraine', 
           'United Kingdom of Great Britain and Northern Ireland',
           'Montenegro', 'Serbia']

df_life = df_life[df_life['TIME'].isin(countries)]
df_life.head()

,TIME,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
1,Belgium,77.9,78.1,78.2,78.3,79.0,79.1,79.5,79.9,79.8,...,81.4,81.1,81.5,81.6,81.7,82.1,80.8,81.9,81.8,82.5
2,Bulgaria,71.6,71.9,72.1,72.3,72.5,72.5,72.7,73.0,73.3,...,74.5,74.7,74.9,74.8,75.0,75.1,73.3,71.2,74.2,75.8
3,Czechia,75.1,75.3,75.4,75.3,75.9,76.1,76.7,77.0,77.3,...,78.9,78.7,79.1,79.1,79.1,79.3,78.3,77.2,79.0,80.0
4,Denmark,76.9,77.0,77.1,77.4,77.8,78.3,78.4,78.4,78.8,...,80.7,80.8,80.9,81.1,81.0,81.5,81.6,81.5,81.3,81.9
5,Germany,78.3,78.6,78.6,78.6,79.3,79.4,79.9,80.1,80.2,...,81.2,80.7,81.0,81.1,81.0,81.3,81.1,80.8,80.7,81.2


Now we clean up the missing values. Since in most of the countries not that many values are missing, we can impute the data. For this we calculate the average change of life expectany in the given country over the observation period and use this to inpute the missing values.

In [78]:
import numpy as np


year_cols = [str(year) for year in range(2000, 2024)]
df_life[year_cols] = df_life[year_cols].replace('...', np.nan)
df_life[year_cols] = df_life[year_cols].apply(pd.to_numeric, errors='coerce')

def impute_country_data(row):
    values = row[year_cols].copy()
    
    # Calculate average annual change
    deltas = []
    for i in range(1, len(values)):
        prev = values[i-1]
        curr = values[i]
        if not np.isnan(prev) and not np.isnan(curr):
            deltas.append(curr - prev)
    
    if not deltas:  # No available deltas
        return row
    
    avg_delta = np.mean(deltas)
    
    # Forward fill
    for i in range(1, len(values)):
        if np.isnan(values[i]) and not np.isnan(values[i-1]):
            values[i] = values[i-1] + avg_delta
    
    # Backward fill
    for i in range(len(values)-2, -1, -1):
        if np.isnan(values[i]) and not np.isnan(values[i+1]):
            values[i] = values[i+1] - avg_delta
    
    row[year_cols] = values
    return row

# Apply imputation to each country
df_imputed = df_life.apply(impute_country_data, axis=1)
df_imputed[year_cols] = df_imputed[year_cols].round(1)
df_imputed.head()

C:\Users\Marci\AppData\Local\Temp\ipykernel_18496\2978272261.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prev = values[i-1]
C:\Users\Marci\AppData\Local\Temp\ipykernel_18496\2978272261.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  curr = values[i]
C:\Users\Marci\AppData\Local\Temp\ipykernel_18496\2978272261.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if np.isnan(values[i]) and not np.isnan(values[i-1]):
C:\U

,TIME,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
1,Belgium,77.9,78.1,78.2,78.3,79.0,79.1,79.5,79.9,79.8,...,81.4,81.1,81.5,81.6,81.7,82.1,80.8,81.9,81.8,82.5
2,Bulgaria,71.6,71.9,72.1,72.3,72.5,72.5,72.7,73.0,73.3,...,74.5,74.7,74.9,74.8,75.0,75.1,73.3,71.2,74.2,75.8
3,Czechia,75.1,75.3,75.4,75.3,75.9,76.1,76.7,77.0,77.3,...,78.9,78.7,79.1,79.1,79.1,79.3,78.3,77.2,79.0,80.0
4,Denmark,76.9,77.0,77.1,77.4,77.8,78.3,78.4,78.4,78.8,...,80.7,80.8,80.9,81.1,81.0,81.5,81.6,81.5,81.3,81.9
5,Germany,78.3,78.6,78.6,78.6,79.3,79.4,79.9,80.1,80.2,...,81.2,80.7,81.0,81.1,81.0,81.3,81.1,80.8,80.7,81.2


Now let's create another df with the same data but in a format that's more compatible with the gdp and trade data

In [79]:
# Melt the dataframe to create year-country-value rows
df_long = df_imputed.melt(
    id_vars=['TIME'], 
    value_vars=year_cols,
    var_name='Year',
    value_name='Life Expectancy'
)

# Clean up and sort
df_long = df_long.rename(columns={'TIME': 'Area'})
df_long['Year'] = df_long['Year'].astype(int)
df_long = df_long.sort_values(['Area', 'Year']).reset_index(drop=True)

# Round to 1 decimal (if not already done)
df_long['Life Expectancy'] = df_long['Life Expectancy'].round(1)

# Show the result
df_long.head()

,Area,Year,Life Expectancy
0,Albania,2000,76.2
1,Albania,2001,76.3
2,Albania,2002,76.4
3,Albania,2003,76.6
4,Albania,2004,76.7


### 2.3 Loading and cleaning gdp per capita dataset

### 2.4 Loading and cleaning food consumption data

## 3. Exploratory analysis

idk add code and texts

## 4. Training models to answer some questions + Eval

When we first looked through the available datasets, we wanted to pick topics that would be both meaningful to analyze and feasible with machine learning. The diabetes risk analysis caught our attention since it connects food consumption patterns to public health outcomes - something that affects millions of people worldwide. For our second topic, we chose to build a regression model for GDP per capita prediction, as it seemed to be an interesting challenge to find some correlation between the food consumption/food related health issues and the economic strength of the countries. Though we hadn't done deep analysis yet, both datasets seemed to have good quality data and enough variables to work with. We basically tried to find a sweet spot between picking important real-world problems and making sure we had enough solid data to actually build useful models.

The first topic is related to the fifth question about the risk of diabetes in relation to food
consumption.

### 4.1 Load the data forthe food related ML shit


In [ ]:
import pandas as pd

life_food = pd.read_csv('./data/final/life_food.csv')
life_food

,geo,TIME_PERIOD,sex,Food Category,Mean,Standard Deviation,5th percentile,10th percentile,Median,95th percentile,97.5th percentile,99th percentile,life_expectancy
0,Austria,2010,Females,Alcoholic beverages,1.947294,3.575471,0.000000,0.000000,0.280000,9.163000,10.997647,13.919059,83.5
1,Austria,2010,Females,Animal and vegetable fats and oils and primary...,0.409974,0.340208,0.032057,0.055547,0.333906,0.984349,1.160260,1.413906,83.5
2,Austria,2010,Females,"Coffee, cocoa, tea and infusions",5.094759,4.539578,0.322410,0.375572,3.945813,13.629217,15.740271,19.433343,83.5
3,Austria,2010,Females,Composite dishes,3.111088,2.397241,0.035172,0.496737,2.744218,7.685544,8.503422,9.751645,83.5
4,Austria,2010,Females,Eggs and egg products,0.322270,0.441656,0.000000,0.000000,0.158466,1.202699,1.291902,1.522515,83.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,United Kingdom,2011,Males,"Seasoning, sauces and condiments",0.528884,0.956948,0.000000,0.000000,0.137267,2.337289,3.032118,4.047449,79.0
2668,United Kingdom,2011,Males,"Starchy roots or tubers and products thereof, ...",2.644138,2.912759,0.000000,0.000000,1.813793,8.179655,9.726207,12.794483,79.0
2669,United Kingdom,2011,Males,"Sugar and similar, confectionery and water-bas...",0.391207,0.850000,0.000000,0.000000,0.074976,1.827308,2.761191,3.705171,79.0
2670,United Kingdom,2011,Males,Vegetables and vegetable products,3.796339,3.931466,0.000000,0.106760,2.805803,11.865891,13.949526,17.654372,79.0


In [7]:
# 2. Group by (TIME_PERIOD, geo) to find the set of sexes for each (year, country)
df_sex_sets = (
    life_food
    .groupby(['TIME_PERIOD', 'geo'])['sex']
    .apply(set)              # e.g. {'Males'}, {'Females'}, or {'Males', 'Females'}
    .reset_index(name='sex_set')
)

# 3. Define a helper function to label the sex category based on the set
def label_sex_category(sex_set):
    if sex_set == {'Males'}:
        return 'Male'
    elif sex_set == {'Females'}:
        return 'Female'
    elif sex_set == {'Males', 'Females'}:
        return 'Combined'
    else:
        # Optional: handle unexpected cases, e.g. empty set or something else
        return 'Other'

df_sex_sets['sex_category'] = df_sex_sets['sex_set'].apply(label_sex_category)

# 4. For each year, gather the countries in each category into lists
result = (
    df_sex_sets
    .groupby('TIME_PERIOD')
    .apply(lambda g: pd.Series({
        'Male': list(g.loc[g['sex_category'] == 'Male', 'geo'].unique()),
        'Female': list(g.loc[g['sex_category'] == 'Female', 'geo'].unique()),
        'Combined': list(g.loc[g['sex_category'] == 'Combined', 'geo'].unique())
    }))
    .reset_index()
)

# 5. (Optional) Rename the TIME_PERIOD column to "Year" for clarity
result.rename(columns={'TIME_PERIOD': 'Year'}, inplace=True)

# Show the final DataFrame
result

C:\Users\Marci\AppData\Local\Temp\ipykernel_13128\500446393.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,Year,Male,Female,Combined
0,1997,[],[],"[Ireland, Sweden]"
1,1998,[],[],[Spain]
2,1999,[],[],[Spain]
3,2000,[],[],"[Denmark, Finland, United Kingdom]"
4,2001,[],[],"[Finland, Germany]"
5,2002,[],[],[Belgium]
6,2003,[],[],"[Cyprus, Czechia, Finland, Hungary, Netherland..."
7,2004,[],[],"[Belgium, Greece, Spain]"
8,2005,[],[Greece],"[Denmark, Italy]"
9,2006,[],[],"[Denmark, Germany, Netherlands]"


In [8]:
# 2. For each (country, year), determine the set of sexes
df_sex_sets = (
    life_food
    .groupby(['geo', 'TIME_PERIOD'])['sex']
    .apply(set)  # e.g. {'Males'}, {'Females'}, or {'Males','Females'}
    .reset_index(name='sex_set')
)

# 3. Filter to keep only the rows where both Males and Females are present
df_combined_only = df_sex_sets[df_sex_sets['sex_set'] == {'Males', 'Females'}]

# 4. Group by 'geo' and collect all years that meet the condition
df_result = (
    df_combined_only
    .groupby('geo')['TIME_PERIOD']
    .apply(list)
    .reset_index(name='years_with_combined_data')
)

# Show the final DataFrame
print(df_result)

               geo              years_with_combined_data
0          Austria                    [2010, 2014, 2018]
1          Belgium                    [2002, 2004, 2014]
2         Bulgaria                                [2007]
3          Croatia                    [2011, 2019, 2021]
4           Cyprus                          [2003, 2014]
5          Czechia                                [2003]
6          Denmark                    [2000, 2005, 2006]
7          Estonia                                [2013]
8          Finland  [2000, 2001, 2003, 2007, 2012, 2017]
9           France                          [2007, 2014]
10         Germany                    [2001, 2006, 2007]
11          Greece                          [2004, 2014]
12         Hungary                          [2003, 2018]
13         Ireland                          [1997, 2008]
14           Italy                    [2005, 2017, 2018]
15          Latvia                          [2008, 2012]
16      Montenegro             

Based on these tables the we can see that the data is not consistent, it is in really bad condition. On the other hand we can select a few countries and years that have more data than otheres, and can try to come up with some new questions that we might be able to answer.
The most interesting countries are:
- Finland
- Netherlands
- Spain
- Sweden

The years with most data are:
- 2003
- 2007
- 2012
- 2017
- 2019

In [5]:
diabetes_food = pd.read_csv('./data/final/diabetes_food.csv')
diabetes_food

,sex,geo,TIME_PERIOD,deaths_out_of_100k,Food Category,Mean,Standard Deviation,5th percentile,10th percentile,Median,95th percentile,97.5th percentile,99th percentile
0,Females,Austria,2014,32.88,Alcoholic beverages,1.230000,2.480000,0.000000,0.000000,0.000000,5.630000,8.210000,11.450000
1,Females,Austria,2014,32.88,Animal and vegetable fats and oils and primary...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Females,Austria,2014,32.88,"Coffee, cocoa, tea and infusions",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Females,Austria,2014,32.88,Composite dishes,0.530000,1.020000,0.000000,0.000000,0.000000,2.560000,3.570000,4.710000
4,Females,Austria,2014,32.88,Eggs and egg products,0.310000,0.440000,0.000000,0.000000,0.110000,1.140000,1.470000,1.910000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1157,Males,United Kingdom,2011,12.56,"Seasoning, sauces and condiments",0.528884,0.956948,0.000000,0.000000,0.137267,2.337289,3.032118,4.047449
1158,Males,United Kingdom,2011,12.56,"Starchy roots or tubers and products thereof, ...",2.644138,2.912759,0.000000,0.000000,1.813793,8.179655,9.726207,12.794483
1159,Males,United Kingdom,2011,12.56,"Sugar and similar, confectionery and water-bas...",0.391207,0.850000,0.000000,0.000000,0.074976,1.827308,2.761191,3.705171
1160,Males,United Kingdom,2011,12.56,Vegetables and vegetable products,3.796339,3.931466,0.000000,0.106760,2.805803,11.865891,13.949526,17.654372


In [6]:
obesity_food = pd.read_csv('./data/final/obesity_food.csv')
obesity_food

,sex,geo,TIME_PERIOD,obesity_percentage,Food Category,Mean,Standard Deviation,5th percentile,10th percentile,Median,95th percentile,97.5th percentile,99th percentile
0,Females,Finland,2017,35.1,Alcoholic beverages,0.773147,2.189442,0.000000,0.000000,0.000000,4.609543,7.046193,10.210305
1,Females,Finland,2017,35.1,"Amphibians, reptiles, and terrestrial inverteb...",0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Females,Finland,2017,35.1,Animal and vegetable fats and oils and primary...,0.357128,0.251384,0.045744,0.085744,0.317128,0.821279,0.992663,1.144047
3,Females,Finland,2017,35.1,"Coffee, cocoa, tea and infusions",7.812036,4.923349,1.339692,2.814485,6.994982,16.543941,20.342935,24.609917
4,Females,Finland,2017,35.1,Composite dishes,0.099432,0.337102,0.000000,0.000000,0.000000,0.620114,0.976648,1.286648
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4015,Males,Slovenia,2017,1.1,"Seasoning, sauces and condiments",0.310197,0.653862,0.008742,0.021171,0.089661,1.451619,2.319562,3.419628
4016,Males,Slovenia,2017,1.1,"Starchy roots or tubers and products thereof, ...",1.626212,1.764161,0.000000,0.000000,1.255065,4.937116,6.032410,6.560990
4017,Males,Slovenia,2017,1.1,"Sugar and similar, confectionery and water-bas...",0.241502,0.446955,0.000000,0.000000,0.053992,1.032160,1.617016,2.113519
4018,Males,Slovenia,2017,1.1,Vegetables and vegetable products,2.771213,2.208702,0.360819,0.542340,2.255436,7.236340,8.304032,9.361936


## 4.2 Load data for the GDP related ML shit

In [14]:
quantity_food = pd.read_csv('./data/final/quantity_food.csv')
quantity_food

,geo,TIME_PERIOD,sex,Food Category,Mean,Standard Deviation,5th percentile,10th percentile,Median,95th percentile,97.5th percentile,99th percentile,Imported quantity,Exported quantity
0,Austria,2010,Females,Alcoholic beverages,1.947294,3.575471,0.000000,0.000000,0.280000,9.163000,10.997647,13.919059,174183.00,152650.00
1,Austria,2010,Females,Animal and vegetable fats and oils and primary...,0.409974,0.340208,0.032057,0.055547,0.333906,0.984349,1.160260,1.413906,525682.00,283082.00
2,Austria,2010,Females,"Coffee, cocoa, tea and infusions",5.094759,4.539578,0.322410,0.375572,3.945813,13.629217,15.740271,19.433343,94578.00,29286.00
3,Austria,2010,Females,Composite dishes,3.111088,2.397241,0.035172,0.496737,2.744218,7.685544,8.503422,9.751645,424667.00,478336.00
4,Austria,2010,Females,Eggs and egg products,0.322270,0.441656,0.000000,0.000000,0.158466,1.202699,1.291902,1.522515,30094.00,8838.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998,Sweden,2016,Males,"Seasoning, sauces and condiments",0.810000,0.970000,0.000000,0.030000,0.470000,2.340000,3.780000,4.210000,42027.15,2409.15
1999,Sweden,2016,Males,"Starchy roots or tubers and products thereof, ...",2.060000,2.750000,0.000000,0.000000,1.390000,6.270000,9.040000,10.770000,323593.12,15602.43
2000,Sweden,2016,Males,"Sugar and similar, confectionery and water-bas...",0.522399,1.592543,0.000000,0.000000,0.000704,2.567787,3.822026,5.849770,232870.30,221299.15
2001,Sweden,2016,Males,Vegetables and vegetable products,2.470000,2.060000,0.410000,0.540000,1.770000,6.420000,7.790000,8.650000,529495.56,227915.41


In [16]:
gdp_food = pd.read_csv('./data/final/gdp_food.csv')
gdp_food

,geo,TIME_PERIOD,sex,Food Category,Mean,Standard Deviation,5th percentile,10th percentile,Median,95th percentile,97.5th percentile,99th percentile,gdp_per_capita
0,Austria,2014,Females,Alcoholic beverages,1.230000,2.480000,0.00,0.00,0.000000,5.630000,8.210000,11.45000,129
1,Austria,2014,Females,Animal and vegetable fats and oils and primary...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129
2,Austria,2014,Females,"Coffee, cocoa, tea and infusions",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129
3,Austria,2014,Females,Composite dishes,0.530000,1.020000,0.00,0.00,0.000000,2.560000,3.570000,4.71000,129
4,Austria,2014,Females,Eggs and egg products,0.310000,0.440000,0.00,0.00,0.110000,1.140000,1.470000,1.91000,129
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179,Sweden,2016,Males,"Seasoning, sauces and condiments",0.810000,0.970000,0.00,0.03,0.470000,2.340000,3.780000,4.21000,123
1180,Sweden,2016,Males,"Starchy roots or tubers and products thereof, ...",2.060000,2.750000,0.00,0.00,1.390000,6.270000,9.040000,10.77000,123
1181,Sweden,2016,Males,"Sugar and similar, confectionery and water-bas...",0.522399,1.592543,0.00,0.00,0.000704,2.567787,3.822026,5.84977,123
1182,Sweden,2016,Males,Vegetables and vegetable products,2.470000,2.060000,0.41,0.54,1.770000,6.420000,7.790000,8.65000,123


In [17]:
gdp_quantity = pd.read_csv('./data/final/gdp_quantity.csv')
gdp_quantity

,geo,TIME_PERIOD,gdp_per_capita,Imported quantity,Exported quantity,Food Category
0,Albania,2012,30,33539.00,1055.00,Alcoholic beverages
1,Albania,2012,30,0.00,0.00,"Amphibians, reptiles, and terrestrial inverteb..."
2,Albania,2012,30,44965.00,1595.00,Animal and vegetable fats and oils and primary...
3,Albania,2012,30,7159.00,33.00,"Coffee, cocoa, tea and infusions"
4,Albania,2012,30,32278.00,1823.00,Composite dishes
...,...,...,...,...,...,...
7263,Slovakia,2022,71,27320.17,2636.52,"Seasoning, sauces and condiments"
7264,Slovakia,2022,71,139305.03,133010.92,"Starchy roots or tubers and products thereof, ..."
7265,Slovakia,2022,71,182229.63,269151.79,"Sugar and similar, confectionery and water-bas..."
7266,Slovakia,2022,71,231619.67,50052.84,Vegetables and vegetable products


## 5. Answer the questions we have lsited, if they are already answered before, then we can just sum it up here

## 6. Conclusions, challenges, fuck knows